In [ ]:
import cv2
import cvzone
import math
from ultralytics import YOLO

cap = cv2.VideoCapture('/home/wani/Downloads/fall.mp4')

model = YOLO('yolov8s.pt')

classnames = []
with open('/home/wani/Downloads/classes.txt', 'r') as f:
    classnames = f.read().splitlines()


while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (980,740))

    results = model(frame)

    for info in results:
        parameters = info.boxes
        for box in parameters:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            confidence = box.conf[0]
            class_detect = box.cls[0]
            class_detect = int(class_detect)
            class_detect = classnames[class_detect]
            conf = math.ceil(confidence * 100)


            # implement fall detection using the coordinates x1,y1,x2
            height = y2 - y1
            width = x2 - x1
            threshold  = height - width

            if conf > 80 and class_detect == 'person':
                cvzone.cornerRect(frame, [x1, y1, width, height], l=30, rt=6)
                cvzone.putTextRect(frame, f'{class_detect}', [x1 + 8, y1 - 12], thickness=2, scale=2)
            
            if threshold < 0:
                cvzone.putTextRect(frame, 'Fall Down!!!', [height, width], thickness=2, scale=2)
            
            else:pass


    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('t'):
        break


cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import cvzone
import math
from ultralytics import YOLO

# 웹캠으로 비디오 캡처 시작 (0번은 기본 웹캠)
cap = cv2.VideoCapture(0)

# YOLO 모델 로드
model = YOLO('yolov8s.pt')

classnames = []
with open('/home/wani/Downloads/classes.txt', 'r') as f:
    classnames = f.read().splitlines()

while True:
    ret, frame = cap.read()
    if not ret:
        print("웹캠 연결 오류")
        break

    # 프레임 크기 조정 (웹캠 해상도에 맞게 조정 가능)
    frame = cv2.resize(frame, (980, 740))

    # 모델을 통해 결과 추론
    results = model(frame)

    # 추론 결과에서 객체 감지
    for info in results:
        parameters = info.boxes
        for box in parameters:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            confidence = box.conf[0]
            class_detect = box.cls[0]
            class_detect = int(class_detect)
            class_detect = classnames[class_detect]
            conf = math.ceil(confidence * 100)

            # 낙상 감지 로직 (사람이 감지되면 감지 박스 표시)
            height = y2 - y1
            width = x2 - x1
            threshold = height - width

            if conf > 80 and class_detect == 'person':  # 사람 클래스에 대해 낙상 감지
                cvzone.cornerRect(frame, [x1, y1, width, height], l=30, rt=6)
                cvzone.putTextRect(frame, f'{class_detect}', [x1 + 8, y1 - 12], thickness=2, scale=2)

                # 낙상 감지: 고관절(y 좌표)이 무릎(y 좌표)보다 낮을 때
                if threshold < 0:
                    cvzone.putTextRect(frame, 'Fall Down!!!', [x1, y2], thickness=2, scale=2)

    # 실시간 영상 출력
    cv2.imshow('Real-time Fall Detection', frame)

    # 't' 키를 눌러서 종료
    if cv2.waitKey(1) & 0xFF == ord('t'):
        break

# 웹캠과 창 닫기
cap.release()
cv2.destroyAllWindows()